In [4]:
import numpy as np
import qutip as qt
from scipy.linalg import logm, expm
from qiskit.quantum_info import Operator, state_fidelity
from qiskit import QuantumCircuit
from qiskit_aer import StatevectorSimulator
from qiskit import Aer
from qiskit.circuit.library import QFT
import pickle
from time import time

from op_fourier_trafo import *
from boltzmann import *
from liouv_step import *
from tools.classical import *
from tools.quantum import *

In [67]:
sigmax = np.array([[0, 1], [1, 0]])
sigmay = np.array([[0, -1j], [1j, 0]])
sigma_minus = (sigmax - 1j * sigmay) / 2
term_coeffs = np.array([0.5, -1j*0.5])
sqrt_term_coeffs = np.sqrt(term_coeffs) #/ np.sqrt(np.sum(term_coeffs))
print(sqrt_term_coeffs)

# Unitary that brings ancilla into right state for LCU 
B = np.array([[sqrt_term_coeffs[0], sqrt_term_coeffs[1].conj()], [sqrt_term_coeffs[1], -sqrt_term_coeffs[0]]])
B_op = Operator(B)

ini_state = np.array([1, 0])
end_state = B @ ini_state
print(end_state)
zerozero = np.array([[1, 0], [0, 0]])
oneone = np.array([[0, 0], [0, 1]])
proj_zero_block = np.kron(np.identity(2), zerozero)
proj_zero_block = np.kron(zerozero, np.identity(2))
proj_one_block = np.kron(np.identity(2), oneone)
print(proj_zero_block)
circ_ini_state = np.array([1, 0, 0, 0])

[0.70710678+0.j  0.5       -0.5j]
[0.70710678+0.j  0.5       -0.5j]
[[1. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 0.]]


In [70]:
num_qubits = 1
num_block_encoding_qubits = 1
qr_sys = QuantumRegister(num_qubits, 'sys')
qr_block = QuantumRegister(num_block_encoding_qubits, 'block')
circ = QuantumCircuit(qr_block, qr_sys)
circ.append(B_op, [qr_block[0]])

circ.h(qr_block)
circ.cx(qr_block, qr_sys)
circ.cy(qr_block, qr_sys)

circ.append(B_op.conjugate().transpose(), [qr_block[0]])
circ_op = Operator(circ)
statevector = Statevector(circ)
print(circ_op.data)


[[ 0.85355339-0.14644661j  0.5       +0.j          0.        +0.j
   0.        +0.j        ]
 [ 0.        -0.5j        -0.14644661+0.85355339j  0.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j          0.35355339-0.35355339j
  -0.70710678+0.5j       ]
 [ 0.        +0.j          0.        +0.j          0.5       -0.70710678j
   0.35355339-0.35355339j]]


In [71]:
proj_state0 = proj_zero_block @ statevector.data
proj_state1 = proj_one_block @ statevector.data
proj_state0

array([0.85355339-0.14644661j, 0.        +0.j        ,
       0.        +0.j        , 0.        +0.j        ])